In [1]:
# Viktor Karsakov 205727183
# Guy Leitersdorfersdorf 201178977

# Question 1

a) Download the "Boston1.csv" database, and explore the data. Explanation about the dataset can be found here: http://www.clemson.edu/economics/faculty/wilson/R-tutorial/analyzing_data.html

Find the columns with missing values and filter them out of the data.

In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

lin = LinearRegression()

# boston = load_boston()
# df = pd.DataFrame(data = boston.data, columns = boston.feature_names)
# df['medv'] = pd.Series(boston.target)
# df["medv"].unique()

df = pd.read_csv("boston1.csv")
df.dropna(axis=1, inplace=True)
df.shape

(506, 15)

b) Divide the filtered data randomly into a train set (70% of the data) and test set (30% of the data).

In [3]:
train, test = train_test_split(df, test_size=0.3)
#test.shape

# Question 2

If you haven't done this previously, install the scikit-learn package for python.

a) On the train set, run a linear regression model as follows:
Divide the training set into explanatory variables (the X matrix with which we'll try to make a prediction) and a target variable (y, the value which we'll try to predict). Use the 'medv' attribute as the target variable y and the rest of the features as the X matrix. Run a linear regression model on those sets, and print the regression coefficients. 

In [4]:
#     print(train.columns.values)
train_cols = list(train.columns.values)
train_cols.remove("medv")

X = train.loc[:,train_cols]
Y = train.loc[:,["medv"]]
Y.shape

test_X = test.loc[:, train_cols]
test_Y = test.loc[:, ["medv"]]
    
res = lin.fit(X,Y)
print("LINEAR REGRESSION COEFFICIENTS: ",res.coef_ )




LINEAR REGRESSION COEFFICIENTS:  [[-1.10754867e-01  4.34751566e-02  4.51336323e-02  3.34670125e+00
  -1.88203513e+01  3.41102081e+00 -1.35332426e-03 -1.46194395e+00
   2.79557321e-01 -1.06912270e-02 -9.77065289e-01  8.70483535e-03
  -5.68348959e-01  5.66375771e-01]]


b) Use the linear regression model to predict the values of the test set's 'medv' column, based on the test set's other attributes. Print the Mean Squared Error of the model on the train set and on the test set.
Usually, the MSE on the train set would be lower than the MSE on the test set, since the model parameters are optimized with respect to the train set. Must this always be the case? Can you think of a few examples for when this might not be the case?

In [5]:
pred_y = lin.predict(X)
pred_test_y = lin.predict(test_X)

MSE_train = mean_squared_error(Y, pred_y)
MSE_test = mean_squared_error(test_Y, pred_test_y)

print("MSE Train = ", MSE_train)
print("MSE Test = ", MSE_test)

#     MSE of the test can be lower than the train in some cases, where the samples of the test are close to the linear 
#     regression surface. 
#     For example, given a surface of a training set, let us look at one point on it, mark it as X. 
#     Now lets assume that the entire test set are samples of the same point X
#     We know that X is predeicted with 100% accuracy (since its on the surface), namely 0 MSE.
#     We also know that not all of the train samples of train set are X, thus the MSE isn't 0.
#     Hence, Test MSE < Train MSE

MSE Train =  24.027665757126535
MSE Test =  17.299262009336775


c) Add some noise (with mean=0, std=1) to the test set's y, and predict it again. What happened to the MSE? Why?

In [6]:
noise = np.random.normal(0,1,[152,1])
noisy_test_Y = test_Y + noise

print("LINEAR REGRESSION WITH NOISE: ", mean_squared_error(noisy_test_Y, pred_test_y))

LINEAR REGRESSION WITH NOISE:  18.496822335868977


# Question 3

a) Create a Recursive feature elimination model, with a linear regression estimator, that selects half of the original number of features. Hint: Check the feature_selection module in scikit-learn.

In [7]:
rfe = RFE(estimator=lin) #,  n_features_to_select=6)

b) Use the feature elimination model on the full database (after filtering columns with missing values, before partitioning into train/test). Print the features that were selected. Remember that we separate the 'medv' attribute to be our y, while the rest of the attributes in the dataset serve as features to learn from.

In [8]:
cols = list(df.columns.values)
# print(cols)
cols.remove("medv")
X_full = df.loc[:,train_cols]
Y_full = df.loc[:,["medv"]]
y_full = np.ravel(Y_full)
rfe.fit(X_full, y_full)
j=0
print("Please Note that the features that were used are the ones set on 'True' ")
for i in cols:
    print(i, " | used: ", rfe.support_[j], "| rank: ", rfe.ranking_[j])
    j+=1

Please Note that the features that were used are the ones set on 'True' 
crim  | used:  False | rank:  3
zn  | used:  False | rank:  5
indus  | used:  False | rank:  4
chas  | used:  True | rank:  1
nox  | used:  True | rank:  1
rm  | used:  True | rank:  1
age  | used:  False | rank:  8
dis  | used:  True | rank:  1
rad  | used:  False | rank:  2
tax  | used:  False | rank:  6
ptratio  | used:  True | rank:  1
black  | used:  False | rank:  7
lstat  | used:  True | rank:  1
randCol  | used:  True | rank:  1


c) We'd like to find out the optimal number of features. Create feature elimination models (with linear regression estimators) for every number of features between 1 and n (where n = all the original features, 'medv' excluded). For each number of features, run a linear regression as in Question 2, only on the selected features, in order to predict 'medv'. Print the Mean Sqaured Error for each number of features.

In [9]:
y = np.ravel(Y)
for i in range(1,14):
    print("==============", i, "==============" )
    rfe = RFE(estimator=lin,  n_features_to_select=i)
    rfe.fit(X_full,np.ravel(Y_full))
    pred_y = rfe.predict(test_X)
    print("MSE - %f" % mean_squared_error(test_Y, np.ravel(pred_y)))
    print("number of features used - %d " % rfe.n_features_)

============== 1 ==============
MSE - 64.617369
number of features used - 1 
============== 2 ==============
MSE - 34.580085
number of features used - 2 
============== 3 ==============
MSE - 32.350879
number of features used - 3 
============== 4 ==============
MSE - 28.111022
number of features used - 4 
============== 5 ==============
MSE - 26.066170
number of features used - 5 
============== 6 ==============
MSE - 19.104018
number of features used - 6 
============== 7 ==============
MSE - 19.103194
number of features used - 7 
============== 8 ==============
MSE - 18.920702
number of features used - 8 
============== 9 ==============
MSE - 18.870616
number of features used - 9 
============== 10 ==============
MSE - 18.605714
number of features used - 10 
============== 11 ==============
MSE - 18.112242
number of features used - 11 
============== 12 ==============
MSE - 17.415735
number of features used - 12 
============== 13 ==============
MSE - 16.611654
number of features us

d) Conclude the optimal number of features for this task. Think about the cost of adding for data vs the benefit of a more accurate prediction. Explain your answer.

In [10]:
print("In conclusion, from examining the above results, we see that the lowest MSE is alternating between 9 features and 12 features due to randomness. we don't get any additional, significant accuracy beyond that. running learning models on a lot of features is expansive, and If we don't gain any more knowledge, it is usually better to choose the minimal number of features. please also note that it also helps us to avoid the curse of dimensionality. there for we choose 9 features, which gives us very low MSE")


In conclusion, from examining the above results, we see that the lowest MSE is alternating between 9 features and 12 features due to randomness. we don't get any additional, significant accuracy beyond that. running learning models on a lot of features is expansive, and If we don't gain any more knowledge, it is usually better to choose the minimal number of features. please also note that it also helps us to avoid the curse of dimensionality. there for we choose 9 features, which gives us very low MSE


# Question 4

Perform a cross-validation of the linear regression on the train set with K=5. Print the CV scores for each repeat.

In [11]:
scores = cross_val_score(lin, X_full,np.ravel(Y_full), cv=5)
print(scores)

[ 0.64016305  0.70491511  0.58645003  0.07720145 -0.27230699]
